In [15]:
# imports
import atc_dataloader, atc_model
import torch
from torch.utils.data import DataLoader
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
# set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class PredictionLoss(nn.Module):
    """
    This class is used to calculate the loss between two vectors
    """
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    mse = nn.MSELoss()
    
    def __init__(self):
        super(PredictionLoss, self).__init__()
    
    def forward(self, pred, target):
        # we are using the mean squared error as loss function
        
    # Define weights for functions for Cos and MSE.
        w1 = 1
        w2 = 10
        
        # Apply cumulative sum to both tensors and calculate loss.
        cos_sim = torch.abs(self.cos(torch.cumsum(pred, dim=-1), torch.cumsum(target, dim=-1))).mean()
        mse_loss = self.mse(torch.cumsum(pred, dim=-1), torch.cumsum(target, dim=-1))
        loss = (w1 * mse_loss) / (w2 * cos_sim)
        return loss

In [9]:
def train_model2(model, device, dataset_train, dataset_test, optimizer, epochs=10):
    
    # define loss function
    criterion = PredictionLoss().to(device)
    
    # create DataLoader for batch processing
    batch_size = 2
    dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    
    losses_overall = []

    # training
    for epoch in range(epochs):
        losses = []
        for (_, batch_in_vectors, batch_out_vectors) in dataloader:
            # get output vector from the model
            print(batch_in_vectors)
            batch_in_vectors = torch.tensor(batch_in_vectors).to(device).float()
            batch_out_vectors = torch.tensor(batch_out_vectors, dtype=float).to(device)
            
            outputs = model(torch.Tensor(batch_in_vectors))
            # forward
            # loss
            loss = criterion(outputs, batch_out_vectors)
            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            
        
        # print out per epoch    
        losses_overall.append(np.mean(losses))
        print(f'Epoch {epoch+1}/{epochs}, Loss: {np.mean(losses)}')

    # plot graph
    plt.plot(np.arange(0,epochs,1),losses_overall)
    plt.title('Loss function')
    plt.show()

In [4]:
# load data
TRAIN_IN="data/train_in.csv"
TRAIN_OUT="data/train_out.csv"

data_train = atc_dataloader.ATCDataset(TRAIN_IN, TRAIN_OUT)

TEST_IN="data/test_in.csv"
TEST_OUT="data/test_out.csv"

data_test = atc_dataloader.ATCDataset(TEST_IN, TEST_OUT)

In [5]:
# testing format of data
a,b,c = data_train.__getitem__(120552)
print(a)
print(c)

12012131
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [23]:
def train_model(model, device, dataloader, criterion, optimizer, epochs=10):
    model.train()  # Set the model to training mode
    
    for epoch in range(epochs):
        total_loss = 0
        # Wrap dataloader with tqdm for progress visualization
        
        for _, batch_in_vectors, batch_out_vectors in tqdm(dataloader, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch'):
            # Convert data to tensors and move to the correct device
            batch_in_vectors = torch.tensor(batch_in_vectors, dtype=torch.float32).to(device)
            batch_out_vectors = torch.tensor(batch_out_vectors, dtype=torch.float32).to(device)
            
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_in_vectors)
            
            # Calculate loss
            loss = criterion(outputs, batch_out_vectors)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Accumulate loss
            total_loss += loss.item()

        # Print loss for the current epoch
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")



In [22]:
model = atc_model.BaseNN().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, weight_decay=0.005)
dataloader = DataLoader(data_train, batch_size=32, shuffle=True)  
criterion = PredictionLoss().to(device)
train_model(model, device, dataloader, criterion, optimizer, epochs=10)
# train_model2(model, device, dataset_train=data_train, dataset_test=data_test, optimizer=optimizer, epochs=10)

Epoch 1/10:   0%|          | 0/3768 [00:00<?, ?batch/s]/tmp/ipykernel_69246/2787234648.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_in_vectors = torch.tensor(batch_in_vectors, dtype=torch.float32).to(device)
/tmp/ipykernel_69246/2787234648.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_out_vectors = torch.tensor(batch_out_vectors, dtype=torch.float32).to(device)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108


Epoch 1/10:   0%|          | 0/3768 [00:08<?, ?batch/s]

109
110


KeyboardInterrupt: 